In [1]:
import json
import math
import numbers
import numpy as np
import pandas as pd


import sys

%load_ext autoreload
%autoreload 2


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# set up display area to show dataframe in jupyter qtconsole

import matplotlib
%matplotlib inline


import matplotlib.pyplot as plt



pd.set_option('max_columns',1000) 
pd.set_option('max_row',300) 
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
import tensorflow as tf

print(tf.__version__)

2.1.0


In [6]:
import tensorflow as tf
import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# 使用 NumPy 生成假数据(phony data), 总共 100 个点.
x_data = np.float32(np.random.rand(2, 100)) # 随机输入
y_data = np.dot([0.100, 0.200], x_data) + 0.300

# 构造一个线性模型
# 
b = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0))
y = tf.matmul(W, x_data) + b

# 最小化方差
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# 初始化变量
init = tf.initialize_all_variables()

# 启动图 (graph)
sess = tf.Session()
sess.run(init)

# 拟合平面
for step in range(0, 201):
    sess.run(train)
    if step % 20 == 0:
        print (step, sess.run(W), sess.run(b))

0 [[0.37017512 0.46947604]] [-0.01016249]
20 [[0.21450463 0.3156077 ]] [0.17369096]
40 [[0.14786695 0.24855441]] [0.24708028]
60 [[0.12004075 0.22036074]] [0.27782696]
80 [[0.10839496 0.20853356]] [0.29070947]
100 [[0.10351721 0.20357591]] [0.29610723]
120 [[0.1014737  0.20149839]] [0.2983689]
140 [[0.10061748 0.20062783]] [0.29931656]
160 [[0.10025872 0.20026305]] [0.29971364]
180 [[0.1001084 0.2001102]] [0.29988003]
200 [[0.10004542 0.20004617]] [0.29994974]


# 计算图
有三种计算图的构建方式：静态计算图，动态计算图，以及Autograph.

在TensorFlow1.0时代，采用的是静态计算图，需要先使用TensorFlow的各种算子创建计算图，然后再开启一个会话Session，显式执行计算图。

而在TensorFlow2.0时代，采用的是动态计算图，即每使用一个算子后，该算子会被动态加入到隐含的默认计算图中立即执行得到结果，而无需开启Session。

使用动态计算图即Eager Excution的好处是方便调试程序，它会让TensorFlow代码的表现和Python原生代码的表现一样，写起来就像写numpy一样，各种日志打印，控制流全部都是可以使用的。

使用动态计算图的缺点是运行效率相对会低一些。因为使用动态图会有许多次Python进程和TensorFlow的C++进程之间的通信。而静态计算图构建完成之后几乎全部在TensorFlow内核上使用C++代码执行，效率更高。此外静态图会对计算步骤进行一定的优化，剪去和结果无关的计算步骤。

如果需要在TensorFlow2.0中使用静态图，可以使用@tf.function装饰器将普通Python函数转换成对应的TensorFlow计算图构建代码。运行该函数就相当于在TensorFlow1.0中用Session执行代码。使用tf.function构建静态图的方式叫做 Autograph.

# 参考 : [eat_tensorflow2_in_30_days/2-2,三种计算图.md](https://github.com/lyhue1991/eat_tensorflow2_in_30_days/blob/master/2-2,%E4%B8%89%E7%A7%8D%E8%AE%A1%E7%AE%97%E5%9B%BE.md)

In [1]:
# 动态计算图在每个算子处都进行构建，构建后立即执行

x = tf.constant("hello")
y = tf.constant("world")
z = tf.strings.join([x,y],separator=" ")

tf.print(z)
hello world
# 可以将动态计算图代码的输入和输出关系封装成函数

def strjoin(x,y):
    z =  tf.strings.join([x,y],separator = " ")
    tf.print(z)
    return z

result = strjoin(tf.constant("hello"),tf.constant("world"))
print(result)
hello world
tf.Tensor(b'hello world', shape=(), dtype=string)

SyntaxError: invalid syntax (<ipython-input-1-36e047526050>, line 8)

In [ ]:
import tensorflow as tf

# 使用autograph构建静态图

@tf.function
def strjoin(x,y):
    z =  tf.strings.join([x,y],separator = " ")
    tf.print(z)
    return z

result = strjoin(tf.constant("你好"),tf.constant("世界"))

print(result)

In [ ]:
tf.__version__

In [17]:
import tensorflow as tf

# f(x) = a*x**2 + b*x + c的导数


x = tf.Variable(0.0,name = "x",dtype = tf.float32)
a = tf.constant(1.0)
b = tf.constant(-2.0)
c = tf.constant(1.0)

@tf.function
def Derivation_of_quadratic(x,a,b,c):
    with tf.GradientTape() as tape:
        y = a*tf.pow(x,2) + b*x + c
    
    dy_dx = tape.gradient(y,x)
    return dy_dx
    
result = Derivation_of_quadratic(x,a,b,c)
print(result)
tf.print(result)
    

tf.Tensor(-2.0, shape=(), dtype=float32)
-2


In [21]:

@tf.function
def f(x):   
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)
    
    # 自变量转换成tf.float32
    x = tf.cast(x,tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(x)
        y = a*tf.pow(x,2)+b*x+c
    dy_dx = tape.gradient(y,x) 
    
    return((dy_dx,y))

tf.print(f(tf.constant(0.0)))
tf.print(f(tf.constant(1.0)))

(-2, 1)
(0, 0)


In [25]:
import tensorflow as tf
import numpy as np 

@tf.function(autograph=True)
def myadd(a,b):
    for i in tf.range(3):
        tf.print(i)
    c = a+b
    print("tracing")
    return c

myadd(1,1)

tracing
0
1
2


<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [ ]:
import datetime

# 创建日志
import os
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = os.path.join('data', 'autograph', stamp)

## 在 Python3 下建议使用 pathlib 修正各操作系统的路径
# from pathlib import Path
# stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# logdir = str(Path('./data/autograph/' + stamp))

writer = tf.summary.create_file_writer(logdir)

#开启autograph跟踪
tf.summary.trace_on(graph=True, profiler=True) 

#执行autograph
result = strjoin("hello","world")
# result = Derivation_of_quadratic(x,a,b,c)
#tf.print(f(tf.constant(0.0)))
#result = myadd(1,1)

#将计算图信息写入日志
with writer.as_default():
    tf.summary.trace_export(
        name="autograph",
        step=0,
        profiler_outdir=logdir)
    


In [ ]:
#启动 tensorboard在jupyter中的魔法命令
%load_ext tensorboard

In [ ]:
#启动tensorboard
%tensorboard --logdir ./data/autograph/